In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data.csv")
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [3]:
df.shape

(10000, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


NLP using spacy

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [16]:
df_small = df[:500]

In [15]:
df["tags"] = df["genre"] +" "+ df["overview"]

In [17]:
df_small.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count', 'tags'],
      dtype='object')

In [18]:
df_small = df_small.drop(["genre","original_language", "popularity", "release_date", "vote_average", "vote_count", "overview"], axis=1)


In [19]:
df_small.head()

,id,title,tags
0,278,The Shawshank Redemption,"Drama,Crime Framed in the 1940s for the double..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance Raj is a rich, carefree, ..."
2,238,The Godfather,"Drama,Crime Spanning the years 1945 to 1955, a..."
3,424,Schindler's List,"Drama,History,War The true story of how busine..."
4,240,The Godfather: Part II,"Drama,Crime In the continuing saga of the Corl..."


In [32]:
X = []
for doc in nlp.pipe(df_small["tags"]):
    X.append(" ".join([t.lemma_ for t in doc if not t.is_stop and not t.is_punct]))

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000)

In [35]:
vector=cv.fit_transform(df_small['tags']).toarray()

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity = cosine_similarity(vector)

In [53]:
def recommand(movies):
    index=df_small[df_small['title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[1:6]:
        print(df_small.iloc[i[0]].title)

In [54]:
recommand("The Shawshank Redemption")

The Truman Show
La luna
Umberto D.
The Lion King
Remi, Nobody's Boy


In [1]:
import joblib
s = joblib.load("similarity.pkl")

In [2]:
s.shape

(9985, 9985)

In [5]:
type(s[0][0])

numpy.float64

In [29]:
import numpy as np

# Assuming arr_float64 is your numpy array of float64
arr_float64 = np.array([1.0, 2.0, 3.0], dtype=np.float64)

# Convert to float32
arr_int = new.astype(np.int8)

# Now arr_float32 is your numpy array of float32
print(arr_float32)


[[1.         0.0541332  0.12888482 ... 0.07319251 0.11065666 0.06277648]
 [0.0541332  1.         0.07325795 ... 0.         0.03494283 0.        ]
 [0.12888482 0.07325795 1.         ... 0.02201127 0.06655584 0.11327309]
 ...
 [0.07319251 0.         0.02201127 ... 1.         0.09449112 0.05360563]
 [0.11065666 0.03494283 0.06655584 ... 0.09449112 1.         0.04052205]
 [0.06277648 0.         0.11327309 ... 0.05360563 0.04052205 1.        ]]


In [8]:
joblib.dump(arr_float32,"small.pkl")

['small.pkl']

In [18]:
len(s)

9985

In [22]:
new = np.empty((9985,9985))

In [25]:
new = np.empty((9985,9985))
for i in range(len(s)):
    l1 = list(set(sorted(s[i])))
    for j in range(len(l1)):
        p = np.where(s[i]==l1[j])[0]
        for k in p:
            new[i][k]=j+1
        
        
        

In [9]:
l1 = np.array([1,2,3,4,5,1])

In [13]:
np.where(l1==1)[0]

array([0, 5])

In [44]:
new.shape

(9985, 9985)

In [45]:
joblib.dump(new,"int.pkl")

['int.pkl']

In [41]:
arr = new.astype(np.uint64)

In [42]:
max(arr[1])

522